## Set up Reviewer expertise
This conference is based on the test_iclr_conference_v2.py, with some simplifications

Some notes: 
- Some code cannot be run more than once without giving an error (for example if you withdraw a paper, then withdraw it again, it will give you an error)
- Most code depends on all of the code before it being run, since this is simulating a compressed conference workflow

In [64]:
from helpers import *
import time
import openreview
import pytest
import datetime
import re
import random
import os
import csv
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
selenium = webdriver.Firefox()
helpers = Helpers()

## Make sample users and create deadline
From: test_create_conference()

This section connects to the client, sets up variables for deadlines, and makes sample profiles for the conference

In [65]:
#Set up deadlines
now = datetime.datetime.utcnow()
abstract_date = now + datetime.timedelta(days=1)
due_date = now + datetime.timedelta(days=3)

#connect to client
client = openreview.Client(baseurl = 'http://localhost:3000', username = 'openreview.net', password=helpers.strong_password)
openreview_client =openreview.Client(baseurl = 'http://localhost:3001', username = 'openreview.net', password=helpers.strong_password)

In [66]:
history = [{
                'position': 'PhD Student',
                'start': 2017,
                'end': None,
                'institution': {
                    'country': 'US',
                    'domain': 'myuniversity.edu',
                    'name' : 'My University'
                }
            }]
example_client = helpers.create_complete_user('test@gmail.com', 'Keith','Johnson',history)
profile_url = 'http://localhost:3030/profile?email='+'test@gmail.com'



In [67]:
profile_url
request_page(selenium, profile_url, by=By.CLASS_NAME, wait_for_element='note_editor',token=example_client.token)


Timed out waiting for page to load


Next, upload DBLP papers: https://dblp.org/pid/73/1654.html

In [ ]:
# dblp_group_id = 'DBLP.org'
# dblp_uploader_group_id = f'{dblp_group_id}/Uploader'
# client.get_i

Look at a profile using this url: http://localhost:3030/profile?email=[EMAIL]

Example: http://localhost:3030/profile?email=pc@mail.com

All profiles use the same password: Or$3cur3P@ssw0rd (also stored in helpers.strong_password)

## Set up conference
From: test_create_conference()

This section sets up the venue request form and deploys it with the settings for the venue. 

In [68]:
# Make PCs
helpers.create_user('pc@mail.com', 'Program','Chair')
pc_client = openreview.Client(username='pc@mail.com', password=helpers.strong_password)

In [69]:
#Post the venue request form and deploty the conference

request_form_note = pc_client.post_note(openreview.Note(
        invitation='openreview.net/Support/-/Request_Form',
        signatures=['~Program_Chair1'],
        readers=[
            'openreview.net/Support',
            '~Program_Chair1'
        ],
        writers=[],
        content={
            'title': 'Test Conference', 
            'Official Venue Name': 'Test Conference', 
            'Abbreviated Venue Name': 'TC 2024',  
            'Official Website URL': 'https//:testconference.com', 
            'program_chair_emails': ['pc@mail.com'],
            'contact_email': 'pc@mail.com', 
            'publication_chairs':'No, our venue does not have Publication Chairs',
            'Area Chairs (Metareviewers)': 'Yes, our venue has Area Chairs',
            'senior_area_chairs': 'No, our venue does not have Senior Area Chairs',
            'Venue Start Date': '2024/07/01',
            'abstract_registration_deadline': abstract_date.strftime('%Y/%m/%d'),
            'Submission Deadline': due_date.strftime('%Y/%m/%d'),
            'Location': 'Virtual',
            'submission_reviewer_assignment': 'Automatic',
            'Author and Reviewer Anonymity': 'Double-blind',
            'reviewer_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'senior_area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'Open Reviewing Policy': 'Submissions and reviews should both be public.',
            'submission_readers': 'Everyone (submissions are public)',
            'withdrawn_submissions_visibility': 'Yes, withdrawn submissions should be made public.',
            'withdrawn_submissions_author_anonymity': 'Yes, author identities of withdrawn submissions should be revealed.',
            'desk_rejected_submissions_visibility':'Yes, desk rejected submissions should be made public.',
            'desk_rejected_submissions_author_anonymity':'Yes, author identities of desk rejected submissions should be revealed.',
            'How did you hear about us?': 'ML conferences',
            'Expected Submissions': '100',
            'use_recruitment_template': 'Yes',
            'api_version': '2',
            'submission_license': ['CC BY 4.0', 'CC BY-SA 4.0', 'CC0 1.0'] # Allow authors to select license
        }))

helpers.await_queue()

# Deploy the conference

client.post_note(openreview.Note(
    content={'venue_id': 'TC/2024/Conference'},
    forum=request_form_note.forum,
    invitation='openreview.net/Support/-/Request{}/Deploy'.format(request_form_note.number),
    readers=['openreview.net/Support'],
    referent=request_form_note.forum,
    replyto=request_form_note.forum,
    signatures=['openreview.net/Support'],
    writers=['openreview.net/Support']
))

helpers.await_queue()


The venue will be shown here: http://localhost:3030/

## Reviewer Recruitment Stage
From: test_reviewer_recruitment()

This section sets up and runs reviewer recruitment and shows a sample response from a reviewer

In [70]:
# Set up reviewer recruitment
pc_client=openreview.Client(username='pc@mail.com', password=helpers.strong_password)
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]

reviewer_details = '''test@gmail.com, Keith Johnson'''
pc_client.post_note(openreview.Note(
    content={
        'title': 'Recruitment',
        'invitee_role': 'Reviewers',
        'invitee_details': reviewer_details,
        'allow_accept_with_reduced_load': 'Yes',
        'invitee_reduced_load': ["1", "2", "3"],
        'invitation_email_subject': 'Invitation to serve as {{invitee_role}}',
        'invitation_email_content': 'Dear {{fullname}},\n\nYou have been nominated to serve as {{invitee_role}}.\n\n{{invitation_url}}\n\nCheers!\n\nProgram Chairs'
    },
    forum=request_form.forum,
    replyto=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Recruitment'.format(request_form.number),
    readers=['TC/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    signatures=['~Program_Chair1'],
    writers=[]
))

helpers.await_queue()

In [71]:
#example response: accept invitation with reduced load
messages = openreview_client.get_messages(subject = 'Invitation to serve as Reviewer')

for message in messages:
    text = message['content']['text']
    invitation_url = re.search('https://.*\n', text).group(0).replace('https://openreview.net', 'http://localhost:3030').replace('&amp;', '&')[:-1]
    print(invitation_url)
    helpers.respond_invitation(selenium, request_page, invitation_url, accept=True, quota=3)

#helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/Reviewers/-/Recruitment', count=2)


http://localhost:3030/invitation?id=TC/2024/Conference/Reviewers/-/Recruitment&user=test%40gmail.com&key=1cf1d58b8baf84b17cfc1e625305924b23a34b352eec65ee12f6838f1ce6bce7


Next do expertise selection + exclude the first three papers

In [72]:
profile_url
request_page(selenium, profile_url, by=By.CLASS_NAME, wait_for_element='note_editor',token=example_client.token)

Timed out waiting for page to load


In [ ]:
# set up the second conference 
#Post the venue request form and deploty the conference
helpers.create_user('pc2@mail.com', 'Program','Chair')
pc_client = openreview.Client(username='pc2@mail.com', password=helpers.strong_password)



request_form_note = pc_client.post_note(openreview.Note(
        invitation='openreview.net/Support/-/Request_Form',
        signatures=['~Program_Chair2'],
        readers=[
            'openreview.net/Support',
            '~Program_Chair2'
        ],
        writers=[],
        content={
            'title': 'Second Test Conference', 
            'Official Venue Name': 'Second Test Conference', 
            'Abbreviated Venue Name': 'TC2 2024',  
            'Official Website URL': 'https//:testconference.com', 
            'program_chair_emails': ['pc2@mail.com'],
            'contact_email': 'pc2@mail.com', 
            'publication_chairs':'No, our venue does not have Publication Chairs',
            'Area Chairs (Metareviewers)': 'Yes, our venue has Area Chairs',
            'senior_area_chairs': 'No, our venue does not have Senior Area Chairs',
            'Venue Start Date': '2024/07/01',
            'abstract_registration_deadline': abstract_date.strftime('%Y/%m/%d'),
            'Submission Deadline': due_date.strftime('%Y/%m/%d'),
            'Location': 'Virtual',
            'submission_reviewer_assignment': 'Automatic',
            'Author and Reviewer Anonymity': 'Double-blind',
            'reviewer_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'senior_area_chair_identity': ['Program Chairs', 'Assigned Senior Area Chair', 'Assigned Area Chair', 'Assigned Reviewers'],
            'Open Reviewing Policy': 'Submissions and reviews should both be public.',
            'submission_readers': 'Everyone (submissions are public)',
            'withdrawn_submissions_visibility': 'Yes, withdrawn submissions should be made public.',
            'withdrawn_submissions_author_anonymity': 'Yes, author identities of withdrawn submissions should be revealed.',
            'desk_rejected_submissions_visibility':'Yes, desk rejected submissions should be made public.',
            'desk_rejected_submissions_author_anonymity':'Yes, author identities of desk rejected submissions should be revealed.',
            'How did you hear about us?': 'ML conferences',
            'Expected Submissions': '100',
            'use_recruitment_template': 'Yes',
            'api_version': '2',
            'submission_license': ['CC BY 4.0', 'CC BY-SA 4.0', 'CC0 1.0'] # Allow authors to select license
        }))

helpers.await_queue()

# Deploy the conference

client.post_note(openreview.Note(
    content={'venue_id': 'TC_2/2024/Conference'},
    forum=request_form_note.forum,
    invitation='openreview.net/Support/-/Request{}/Deploy'.format(request_form_note.number),
    readers=['openreview.net/Support'],
    referent=request_form_note.forum,
    replyto=request_form_note.forum,
    signatures=['openreview.net/Support'],
    writers=['openreview.net/Support']
))

helpers.await_queue()


In [77]:
# Set up reviewer recruitment
request_form=pc_client.get_notes(invitation='openreview.net/Support/-/Request_Form')[0]


reviewer_details = '''test@gmail.com, Keith Johnson'''
pc_client.post_note(openreview.Note(
    content={
        'title': 'Recruitment',
        'invitee_role': 'Reviewers',
        'invitee_details': reviewer_details,
        'allow_accept_with_reduced_load': 'Yes',
        'invitee_reduced_load': ["1", "2", "3"],
        'invitation_email_subject': 'Invitation to serve as {{invitee_role}}',
        'invitation_email_content': 'Dear {{fullname}},\n\nYou have been nominated to serve as {{invitee_role}}.\n\n{{invitation_url}}\n\nCheers!\n\nProgram Chairs'
    },
    forum=request_form.forum,
    replyto=request_form.forum,
    invitation='openreview.net/Support/-/Request{}/Recruitment'.format(request_form.number),
    readers=['TC_2/2024/Conference/Program_Chairs', 'openreview.net/Support'],
    signatures=['~Program_Chair2'],
    writers=[]
))

helpers.await_queue()

In [80]:
#example response: accept invitation with reduced load
messages = openreview_client.get_messages(subject = 'Invitation to serve as Reviewer')

for message in messages:
    text = message['content']['text']
    invitation_url = re.search('https://.*\n', text).group(0).replace('https://openreview.net', 'http://localhost:3030').replace('&amp;', '&')[:-1]
    print(invitation_url)
    try:
        helpers.respond_invitation(selenium, request_page, invitation_url, accept=True, quota=3)
    except:
        pass

#helpers.await_queue_edit(openreview_client, invitation='TC/2024/Conference/Reviewers/-/Recruitment', count=2)


http://localhost:3030/invitation?id=TC/2024/Conference/Reviewers/-/Recruitment&user=test%40gmail.com&key=1cf1d58b8baf84b17cfc1e625305924b23a34b352eec65ee12f6838f1ce6bce7
http://localhost:3030/invitation?id=TC_2/2024/Conference/Reviewers/-/Recruitment&user=test%40gmail.com&key=1cf1d58b8baf84b17cfc1e625305924b23a34b352eec65ee12f6838f1ce6bce7


In [81]:
profile_url
request_page(selenium, profile_url, by=By.CLASS_NAME, wait_for_element='note_editor',token=example_client.token)

Timed out waiting for page to load
